In [82]:
import pandas as pd
import math
from anytree import Node, RenderTree

In [83]:
#Evaluates a sample using the given decision tree dt. Returns true if the decision tree gets it right, false if not
def evalSample(dt, sample):   
    while (len(dt) == 2):
        dt = dt[1][sample[dt[0]]]
    return (dt[0] == sample['party'])

infileTrain = pd.read_csv("train.csv") #training set
infilePrune = pd.read_csv("prune.csv") #prune set
infileTest = pd.read_csv("test.csv") #testing set


In [84]:
#Just a dummy assingment
df = infileTrain 
#Dummy Decision Tree, ID3 should create your own decision tree.This is a nested representation. Each node is a list. 
#if the node has one entry then it is a leaf node and performs the classification. Otherwise the node has two entries:
#the first entry is the feature ('bruises') and the second entry is a dictionary containing the tree below this node
#for example, bruises has two different possible values ('f' and 't'), therefore the dictionary has those two values
#'t' goes to a leaf node classifying 'p'. However, the 'f' branch has more decision making in this case 'gill-spacing' 
#that all lead to terminal nodes.  
dt = ['physicianFeeFreeze', {'y': ['exportsSouthAfrica', {'y': ['republican'], 
                                         'n': ['democrat'], 
                                         'noVote': ['republican']}], 
                  'n': ['democrat'],
                  'noVote': ['republican']}] 


In [85]:
#Your ID3 algorithm goes here! Use evalSample to evaluate your completed tree on both training and testing sets.
df

,party,handicappedInfants,waterProject,budgetResolution,physicianFeeFreeze,elSalvadorAid,religiousGroupsInSchools,antiSatelliteTestBan,aidToNicaraguanContras,mxMissile,immigration,synfuelsCorporationCutback,educationSpending,superfundRightToSue,crime,dutyFreeExports,exportsSouthAfrica
0,republican,n,y,n,y,y,y,n,n,n,y,noVote,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,noVote
2,democrat,noVote,y,y,noVote,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,noVote,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,noVote,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,republican,n,n,n,y,y,y,n,n,n,y,noVote,y,y,y,n,n
100,democrat,n,n,y,n,n,y,y,noVote,y,y,y,n,n,n,y,y
101,democrat,y,y,noVote,y,y,y,n,n,y,n,y,noVote,y,y,n,n
102,democrat,noVote,y,n,n,n,n,y,y,y,y,y,n,n,y,y,y


In [86]:
#evaluates the 7th sample (0-index) in the training set using the dummy decision tree above 
print(evalSample(dt, df.iloc[1])) 
print(df.iloc[1])


True
party                         republican
handicappedInfants                     n
waterProject                           y
budgetResolution                       n
physicianFeeFreeze                     y
elSalvadorAid                          y
religiousGroupsInSchools               y
antiSatelliteTestBan                   n
aidToNicaraguanContras                 n
mxMissile                              n
immigration                            n
synfuelsCorporationCutback             n
educationSpending                      y
superfundRightToSue                    y
crime                                  y
dutyFreeExports                        n
exportsSouthAfrica                noVote
Name: 1, dtype: object


In [88]:
from math import log

#compute entropy
def entropy(examples):
    numAttributes = len(examples)
    featureCounts ={}
    for example in examples:
        currentLable=example[-1] 
        if currentLable not in featureCounts.keys():
            featureCounts[currentLable]=0
        featureCounts[currentLable]+=1
    result=0.0
    for i in featureCounts:
        p = float(featureCounts[i]) / numAttributes
        result -= p * log(p, 2)
    return result


def splitData(examples,feat,value):
    subSet=[]
    #return the subset that does not contain values in traget feature
    for featVec in examples:
        #print(featVec[axis])
        if featVec[feat]==value : 
            reduced=featVec[:feat]
            reduced.extend(featVec[feat+1:])
            subSet.append(reduced)
    return subSet 


def BestFeature(examples):
    num=len(examples[0])-1
    print(num)
    bestFeature=-1
    InfoGain=0
    #H(S)
    HEntropy=entropy(examples)
    for i in range(num):
        featList=[example[i] for example in examples] #GET VALUES AT FRETURE[i]
        uniqlVals=set(featList) #SET UNIQUE VALUES
        cEntropy=0
        for value in uniqlVals:
            subDataSet=splitData(examples,i,value)
            prob=len(subDataSet)/float(len(examples)) #p(t)
            cEntropy+=prob*entropy(subDataSet) #H(S/A)    
        infoGain=HEntropy-cEntropy #IG
        #print(infoGain)
        if (infoGain>InfoGain):
            InfoGain=infoGain
            bestFeature=i
    return bestFeature 

In [89]:
def createDataSet(example):
    #rearrange the dataframe and convert to list
    data = example[[c for c in df if c not in ['physicianFeeFreeze']] + ['physicianFeeFreeze']]
    dataSet= data.values.tolist()
    labels= data.columns.tolist()
    return dataSet,labels

In [90]:
def ID3(dataSet,Feature):
    #If all examples are in the same class
    classList=[example[-1] for example in dataSet]
    #print(dataSet[0])
    if classList.count(classList[-1])==len(classList):
        return classList[-1]
    #If Examples is empty,Then below this new branch return most common target value in the examples
    if len(dataSet[0]) == 1:
        return max(set(classList), key = classList.count)  
    bestFeat=BestFeature(dataSet)
    #print(bestFeat,Feature)
    bestFeatLabel=Feature[bestFeat] 
    #print(bestFeatLabel)
    Tree={bestFeatLabel:{}} 
    #print(Feature[bestFeat])
    Feature.remove(bestFeatLabel)
    subLabels=Feature[:]
    #print(subLabels)
    featValues=[example[bestFeat] for example in dataSet]
    uniqVals=set(featValues)
    ##print(uniqueVals)
    #For each possible value of A, Add a new tree branch below
    for value in uniqVals:
        Tree[bestFeatLabel][value]=ID3(splitData(dataSet,bestFeat,value),subLabels)
    return Tree

In [91]:
dataset, datalabels = createDataSet(df)

idt = ID3(dataset,datalabels[:-1])
print(idt)

16
15
14
13
12
14
{'party': {'democrat': {'exportsSouthAfrica': {'y': {'handicappedInfants': {'y': 'n', 'noVote': 'n', 'n': {'budgetResolution': {'y': {'aidToNicaraguanContras': {'y': 'y', 'noVote': 'n', 'n': 'n'}}, 'noVote': 'n', 'n': 'y'}}}}, 'noVote': 'n', 'n': {'waterProject': {'y': 'y', 'noVote': 'noVote', 'n': 'y'}}}}, 'republican': 'y'}}


In [92]:
#classifier based on genreated decision tree and validation set
def classify(idt,Labels,testVec):
    classLabel = {}
    #get first attribute
    firstAttribute=list(idt.keys())[0]
    #create sec dic
    secondDict=idt[firstAttribute] 
    #get index
    Index=Labels.index(firstAttribute) 
    for key in secondDict.keys():
        if testVec[Index]==key:
            if type(secondDict[key]).__name__=='dict':
                classLabel=classify(secondDict[key],Labels,testVec)
            else:classLabel=secondDict[key]
    return classLabel

In [93]:
#Get prediction results
def predict(idt,testdata):
    testdata, testlabels = createDataSet(testdata)
    #classify(idt,testlabels,testdata)
    classes = []
    for i in range(len(testdata)):
        a = classify(idt,testlabels,testdata[i])
        classes.append(a)
    return classes

In [94]:
#compute the accuracy of predictions
def accuracy(idt,val):
    res = predict(idt,val)
    validation = val.iloc[:,5].values.tolist()
    count = 0
    for i in range(len(res)):
        #print(res[i]+' '+validation[i])
        if res[i] == validation[i]:
            count += 1
    return count/len(res)
    

In [95]:
#Accuracy on validataion set(test.csv)
accuracy(idt,infileTest)

0.7014925373134329

In [96]:
#Accuracy on trainning set(test.csv)
accuracy(idt,infileTrain)

0.875